# Data visualization and budget set up

import packages

In [5]:
import sys
sys.path.append(str('/home/jupyter/GroupM_US_Offshore_Hub_Savant/All Regions/'))

from configparser import ConfigParser

from repo.MyConfigParser import MyConfigParser

from repo.LoadData import DataLoader
from repo.BuildModel import GenerateModelFile, CreateModel
from repo.Accuracy import AccuracyCalculator, ProphetParameterSearch, PlotAccuracyCharts
from repo.FileGenerator import *

In [6]:
pd.set_option('display.float_format', '{:.6f}'.format)


initial config file to config_dict

In [7]:
updated_config = ConfigParser()
config_in_path = "/home/jupyter/GroupM_US_Offshore_Hub_Savant/All Regions/emea/June 2024 Update/config ups emea.ini"
updated_config.read(config_in_path)

getdict = MyConfigParser()
config_dict=getdict.GetDict(updated_config)
region = config_dict['CLIENTINFO']['country']
cutoff_month = config_dict['PROPHETSETTING']['cut_off_point']
base_start = config_dict['PROPHETSETTING']['reference_start_date1']
base_end = config_dict['PROPHETSETTING']['reference_end_date1']
periods = config_dict['PROPHETSETTING']['periods']
base_periods  = config_dict['PROPHETSETTING']['base_periods']

training_query = config_dict["PROPHETSETTING"]["training_query"]
_input = config_dict["PROPHETSETTING"]["input"]
_output = config_dict["PROPHETSETTING"]["output"]

output_bucket = config_dict["FILEPATH"]["output_bucket"]
folder_name = config_dict["FILEPATH"]["folder_name"]


today = datetime.date.today().strftime("%m.%d.%Y") #'03.10.2023' #
print(today)
base_path = config_dict["FILEPATH"]["base_path"]

boundary_df_in_path = base_path + config_dict["FILEPATH"]["boundary_df_in_path"]
daily_df_out_path = base_path + config_dict["FILEPATH"]["daily_df_out_path"]
analytical_file_path = base_path + config_dict["FILEPATH"]["analytical_file_path"]
cuts_df_in_path = base_path + config_dict["FILEPATH"]["cuts_df_in_path"]
plot_name = base_path + config_dict["FILEPATH"]["plot_name"]
constraint_file_name = base_path + config_dict["FILEPATH"]["constraint_file_name"]

04.30.2024


Query daily data

In [22]:
daily_df = GetBQData(training_query, _input, _output)
daily_df['month'] = daily_df.ds.dt.month
daily_df['year'] = daily_df.ds.dt.year


/home/jupyter/All Regions/repo/QueryingData.py:34: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  search_df.fillna(method = 'ffill', inplace = True)


In [23]:
daily_df.ds.max()

Timestamp('2024-04-29 00:00:00')

In [24]:
daily_df.market.unique()

array(['Belgium', 'Czech', 'France', 'Germany', 'India', 'Ireland',
       'Italy', 'Netherlands', 'Poland', 'Saudi Arabia', 'Spain',
       'Sweden', 'Turkey', 'UAE', 'UK'], dtype=object)

In [25]:
pre_optimization_budget = daily_df[['year','X']].loc[(daily_df.month == 5) & (daily_df.year >= 2020)].groupby(['year']).sum().mean()


In [26]:
pre_optimization_budget

X   461955.472929
dtype: float64

In [27]:
df = daily_df.copy()
df = df.groupby(['region', 'market', 'account_type', 'funnel', 'ds']).sum().reset_index()


In [28]:
# remove rows without date or conversion quantity
df = df[~df[['ds', 'y']].isin([np.nan, np.inf, -np.inf]).any(axis=1)]


In [29]:
df

,region,market,account_type,funnel,ds,X,y,month,year
0,Europe,Belgium,Bing Ads,Brand,2018-10-05,13.910000,1.000000,10,2018
1,Europe,Belgium,Bing Ads,Brand,2018-10-08,11.760000,1.000000,10,2018
2,Europe,Belgium,Bing Ads,Brand,2018-10-09,19.040000,3.000000,10,2018
3,Europe,Belgium,Bing Ads,Brand,2018-10-10,12.260000,1.000000,10,2018
4,Europe,Belgium,Bing Ads,Brand,2018-10-11,18.020000,2.000000,10,2018
...,...,...,...,...,...,...,...,...,...
106700,Europe,UK,Google AdWords,Generic,2024-04-25,47.188900,2.000000,4,2024
106701,Europe,UK,Google AdWords,Generic,2024-04-26,9.930000,4.000000,4,2024
106702,Europe,UK,Google AdWords,Generic,2024-04-27,0.000000,0.000000,4,2024
106703,Europe,UK,Google AdWords,Generic,2024-04-28,0.000000,3.000000,4,2024
